In [25]:
# General purpose imports to handle paths, files etc
import os
import os.path as op
import glob
import pandas as pd
import numpy as np
import json


# Useful functions to define and import datasets from open neuro
import openneuro
from mne.datasets import sample
from mne_bids import BIDSPath, read_raw_bids, print_dir_tree, make_report


# Useful imports to define the direct download function below
import requests
import urllib.request
from tqdm import tqdm


# FSL function wrappers which we will call from python directly
from fsl.wrappers import fast, bet
from fsl.wrappers.misc import fslroi
from fsl.wrappers import flirt



def reset_overlays():
    """
    Clears view and completely remove visualization. All files opened in FSLeyes are closed.
    The view (along with any color map) is reset to the regular ortho panel.
    """
    l = frame.overlayList
    while(len(l)>0):
        del l[0]
    frame.removeViewPanel(frame.viewPanels[0])
    # Put back an ortho panel in our viz for future displays
    frame.addViewPanel(OrthoPanel)
    
def mkdir_no_exist(path):
    if not op.isdir(path):
        os.makedirs(path)
        
class DownloadProgressBar(tqdm):
    def update_to(self, b=1, bsize=1, tsize=None):
        if tsize is not None:
            self.total = tsize
        self.update(b * bsize - self.n)


def download_url(url, output_path):
    with DownloadProgressBar(unit='B', unit_scale=True,
                             miniters=1, desc=url.split('/')[-1]) as t:
        urllib.request.urlretrieve(url, filename=output_path, reporthook=t.update_to)

def direct_file_download_open_neuro(file_list, file_types, dataset_id, dataset_version, save_dirs):
    # https://openneuro.org/crn/datasets/ds004226/snapshots/1.0.0/files/sub-001:sub-001_scans.tsv
    for i, n in enumerate(file_list):
        subject = n.split('_')[0]
        download_link = 'https://openneuro.org/crn/datasets/{}/snapshots/{}/files/{}:{}:{}'.format(dataset_id, dataset_version, subject, file_types[i],n)
        print('Attempting download from ', download_link)
        download_url(download_link, op.join(save_dirs[i], n))
        print('Ok')
        
def get_json_from_file(fname):
    f = open(fname)
    data = json.load(f)
    f.close()
    return data

In [27]:
#loading the data
dataset_id = 'ds000102'
subject = '01' 

# create dataset folder
sample_path = "dataset"
mkdir_no_exist(sample_path)
bids_root = op.join(os.path.abspath(""),sample_path, dataset_id)
deriv_root = op.join(bids_root, 'derivatives')
preproc_root = op.join(bids_root, 'derivatives','preprocessed_data')

mkdir_no_exist(bids_root)


func_path = op.join(bids_root, 'sub-001', 'func')
anat_path = op.join(bids_root, 'sub-001', 'anat')
mkdir_no_exist(op.join(bids_root, 'sub-001'))
mkdir_no_exist(func_path)
mkdir_no_exist(anat_path)

direct_file_download_open_neuro(file_list=['sub-01_task-flanker_run-1_bold.nii.gz', 
                                           'sub-01_task-flanker_run-2_bold.nii.gz',
                                           'sub-01_T1w.nii.gz'], 
                                file_types=['func', 'func', 'anat'], 
                                dataset_id=dataset_id, 
                                dataset_version='00001', 
                                save_dirs=[func_path,
                                           func_path,
                                           anat_path])

mkdir_no_exist(op.join(bids_root, 'derivatives'))
preproc_root = op.join(bids_root, 'derivatives','preprocessed_data')
mkdir_no_exist(preproc_root)
mkdir_no_exist(op.join(preproc_root, 'sub-01'))
mkdir_no_exist(op.join(preproc_root, 'sub-01', 'anat'))
mkdir_no_exist(op.join(preproc_root, 'sub-01', 'func'))
mkdir_no_exist(op.join(preproc_root, 'sub-01', 'fmap'))

Attempting download from  https://openneuro.org/crn/datasets/ds000102/snapshots/00001/files/sub-01:func:sub-01_task-flanker_run-1_bold.nii.gz


sub-01:func:sub-01_task-flanker_run-1_bold.nii.gz: 28.1MB [00:29, 956kB/s]                               


Ok
Attempting download from  https://openneuro.org/crn/datasets/ds000102/snapshots/00001/files/sub-01:func:sub-01_task-flanker_run-2_bold.nii.gz


sub-01:func:sub-01_task-flanker_run-2_bold.nii.gz: 28.1MB [00:15, 1.84MB/s]                              


Ok
Attempting download from  https://openneuro.org/crn/datasets/ds000102/snapshots/00001/files/sub-01:anat:sub-01_T1w.nii.gz


sub-01:anat:sub-01_T1w.nii.gz: 10.6MB [00:01, 5.73MB/s]                            

Ok
